**In this notebook, we use openai models to rate our results**

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.5 MB/s eta 0:00:00


In [ ]:
import openai
import numpy as np
import tqdm
# fill the api key
openai.api_key = "sk-"

**Define the function, given a question and relevent response, return a score of some aspect from 0 to 10**

In [ ]:
def score_by_openai(input,model="text-davinci-003"):
    prompt = f"Please score the following response to the question based on logic coherence on a scale from 1 to 10:\n\n{input}\n\nScore:"
    response = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.5,
    )
    print(response.choices[0]["text"])
    time.sleep(0.7)
    try:
        numbers = re.findall(r'\d+\.\d+|\d+', response.choices[0]["text"])
        print(numbers)
        score = int(numbers[0])
        if 0 <= score <= 10:
            return score
        else:
            return "The score generated by the model is out of range. Please try again."
    except ValueError:
        return "The model did not generate a valid score. Please try again."



In [ ]:
import re
import time
def score_by_openai_2(input,model="gpt-3.5-turbo"):

    response = openai.ChatCompletion.create(model = model,
                                            messages = [{"role": "system", "content": "Provide a numerical score from 0 to 10 for the reasoning quality of the response to the question given below:"},
                                                         {"role": "user", "content": """{}""".format(input)}],temperature = 1,max_tokens=100  )
    time.sleep(0.5)
    print(response)
    """
    try:
        numbers = re.findall(r'\d+\.\d+|\d+', response.choices[0]["message"]["content"])
        print(numbers)
        score = int(numbers[0])
        if 0 <= score <= 10:
            return score
        else:
            return "The score generated by the model is out of range. Please try again."
    except ValueError:
        return "The model did not generate a valid score. Please try again."
    """
    numbers = re.findall(r'\d+\.\d+|\d+', response.choices[0]["message"]["content"])
    print(numbers)
    if numbers:
        score = float(numbers[0])
        # Clamp the score to the range [0, 10]
        return max(0, min(score, 10))

    # If no number was found or other issues occurred, return a default value (e.g., 5)
    return 5

input = "Question: 1+1=? Response: 1+1=0 if mod 2, so the answer is 0"
score = score_by_openai_2(input)
print(f"score: {score}")

{
  "id": "chatcmpl-7rRSmSZ5zL7hHEZLv8LEEIObBXjSs",
  "object": "chat.completion",
  "created": 1692971308,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "8"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 60,
    "completion_tokens": 1,
    "total_tokens": 61
  }
}
['8']
score: 8.0


**Load the responses**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
folder_path='/content/drive/MyDrive/CoTRL/data/'




Mounted at /content/drive/


In [ ]:
import pandas as pd


df_svamp_base = pd.read_csv(folder_path+'test_base_svamp.csv')
df_svamp_3000 = pd.read_csv(folder_path+'test_3000_svamp.csv')
# Display the first few rows of the DataFrame


In [ ]:
questions_svamp=df_svamp_base["Question"].to_numpy()

repsonses_svamp_base=df_svamp_base["Response"].to_numpy()
repsonses_svamp_3000=df_svamp_3000["Response"].to_numpy()

repsonses_svamp_3000

array(['Thought: \nMary already used 5 cups of flour.\nThe recipe needs 6 - 5 = 1 cup of flour.\nFor sugar, the recipe needs 8 - 5 = 3 cups of sugar.\nFor salt, the recipe needs 7 - 5 = 2 cups of salt.\n#Answer: 3 (cups of sugar) and 2 (cups of salt)',
       '\nThought:\nPaul had 492 crayons initially.\nHe gave away 52 crayons.\nSo 492 - 52 = 440 left.\nLost 535 means Paul lost 535 - 440 = 95 crayons.\n#Answer: 95',
       '\nThought:\nThere are three steps to consider:\n\nStep 1: Robin had 16 inches of hair before cutting. (16 inches)\nStep 2: He cut off 11 inches of hair. (Now 16 - 11 = 5 inches remaining)\nStep 3: His hair grew by 12 inches. (Adding 12 inches to 5 inches, his new length becomes 17 inches)\n\n#Answer: 17 inches',
       "Let's think this through step-by-step! 🤔\n\nOriginally, Josh had 15 marbles in his collection. 🎲\nThen, he found 9 more marbles! 🎉\nSo, the updated number of marbles Josh has is... (check calculator)...15 + 9 = 24! 😃\nNow, let's find out how many ma

**Rate the generated responses to questions in numpy arrays**

In [ ]:
scores_svamp_base=np.zeros(len(questions_svamp))
scores_svamp_3000=np.zeros(len(questions_svamp))

**Using the model: gpt-3.5 turbo**

In [ ]:
for i in tqdm.tqdm(range(len(questions_svamp))):
    input_i_base='Question: '+questions_svamp[i] + ' Response: '+repsonses_svamp_base[i]
    input_i_3000='Question: '+questions_svamp[i] + ' Response: '+repsonses_svamp_3000[i]
    score_i_base=score_by_openai_2(input=input_i_base)
    score_i_3000=score_by_openai_2(input=input_i_3000)
    scores_svamp_base[i]=score_i_base
    scores_svamp_3000[i]=score_i_3000


  0%|          | 0/300 [00:00<?, ?it/s]

{
  "id": "chatcmpl-7rRSxJBwkyFvej7zpGJbtIU3tSPWq",
  "object": "chat.completion",
  "created": 1692971319,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "9"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 234,
    "completion_tokens": 1,
    "total_tokens": 235
  }
}
['9']


  0%|          | 1/300 [00:01<08:23,  1.68s/it]

{
  "id": "chatcmpl-7rRSyzY2BuZcLax3esTUrbPSoTfeG",
  "object": "chat.completion",
  "created": 1692971320,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "8"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 175,
    "completion_tokens": 1,
    "total_tokens": 176
  }
}
['8']
{
  "id": "chatcmpl-7rRSyGY5dwrGuVP0BV95Wky6gZzrf",
  "object": "chat.completion",
  "created": 1692971320,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "8"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 154,
    "completion_tokens": 1,
    "total_tokens": 155
  }
}
['8']


  1%|          | 2/300 [00:03<08:10,  1.65s/it]

...

{
  "id": "chatcmpl-7rRVxwr4dKxxG6sUHxlsYZc8KdL62",
  "object": "chat.completion",
  "created": 1692971505,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "9"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 144,
    "completion_tokens": 1,
    "total_tokens": 145
  }
}
['9']
{
  "id": "chatcmpl-7rRVyg9sQkWQQ7exNgspBs5uFk0z0",
  "object": "chat.completion",
  "created": 1692971506,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "7"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 167,
    "completion_tokens": 1,
    "total_tokens": 168
  }
}
['7']


 33%|███▎      | 98/300 [03:08<05:57,  1.77s/it]

{
  "id": "chatcmpl-7rRVzWCMjZH1kSUyjPxaWpiIh5mLL",
  "object": "chat.completion",
  "created": 1692971507,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "9"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 132,
    "completion_tokens": 1,
    "total_tokens": 133
  }
}
['9']
{
  "id": "chatcmpl-7rRVzE3sWURXgCSTpMZyo4EVIEU09",
  "object": "chat.completion",
  "created": 1692971507,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "5"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 126,
    "completion_tokens": 1,
    "total_tokens": 127
  }
}
['5']


 33%|███▎      | 98/300 [13:09<27:07,  8.06s/it]


APIError: ignored

In [ ]:
print(scores_svamp_base[:98].mean())
print(scores_svamp_3000[:98].mean())

7.982142857142857
8.096938775510203


**Save the rating to a csv document**

In [ ]:
dict = {"type_of_score_aspect":scores_responses}

df_1 = pd.DataFrame(dict)

drive.mount('/content/drive')
path = '/content/drive/My Drive/ratings.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_1.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_1.mean()
df_1.std()

NameError: ignored

# a batch of input

In [ ]:
!pip install openai

In [ ]:
# !ssh-keygen -t ed25519 -C "zy_zhao2021@163.com"
!ssh-keygen -t rsa -b 4096

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 

In [ ]:
!cat /root/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDQq7phjIg3jENVBuVngMQoKwAlt+iUIOWplvk0nxxInebflxsZfx0zUl0/bB5egnOAKX2qYgR1xD/vzzxn+o27BO4ujSrOMqhOdVVVy/tqzCd5SQGmxf79FycGvL1K/QC0rizUMF9Fr6dh0AhRbVGx8T/IfdNUx7TEVKi4o0jLTjmDKXzwazIXC/b7OyYDhPPGG96bwQySVDat3LAzQaOeEFl8X3CNhJC6rIVpWUxc+5veQxQeKucJXMDy7V/vDPxISO8vbnTnAnBc9gbs2WSW2jPpMLg8umoWgj3hvCKtCDZ/2P1liuCq2GI6R4kJPvHUtWoDxilhgLe08TSjnYiPZses1IMAXgiUdZY9Dw1gprJtGXpTyQnjIf1NSUS4uQWVnLnIZLfw5kbFblwDPQoCErMCehrxvyfyIoNGrlyJF9c/JQrSA91vN330n6LEpKnoXAOCuyP7K3rgp3rxsQnvqaTZ8P6jD5Hi6+icehpd3JlJKMWFQdJmg2UwelT/ltZ+cGbvr2MoZ5N7NLpuFN1KfY2ZauhQgnHp2HyrE0NL/Dbz/w8p7vsGu/lPSxxNPt4g3Be7p42OlP5zHgt69oOF9pWQq3Z2clKa23X4L73xGcYBcBKZoAJ+7lFTlhvQfrIxfX0x1rZG8ubhgbmIv9Do/uAHHZ9yHhOEB5WQoMYYGw== root@83c8c5fecd66


In [ ]:
!ssh -T git@github.com
# ! ssh -T -p 443 git@ssh.github.com

Host key verification failed.


In [ ]:
!git clone git@github.com:wuhuaguocao/CoTRL.git

Cloning into 'CoTRL'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 6), reused 11 (delta 1), pack-reused 0
Receiving objects: 100% (21/21), 89.60 KiB | 3.20 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
import pandas as pd

df = pd.read_csv("/content/CoTRL/test_base_svamp.csv")
df.iloc[0]["Question"] + df.iloc[0]["Response"]

FileNotFoundError: ignored

In [ ]:
import re
import time
import openai
def score_by_openai_2(input,model="gpt-3.5-turbo"):

    response = openai.ChatCompletion.create(model = model,messages = [{"role": "system", "content": "Provide a numerical score from 0 to 10 for the commonsense of the response to the question given below, regardless of its completeness or context:"}, {"role": "user", "content": """{}""".format(input)}],temperature = 1,max_tokens=100)
    time.sleep(0.5)
    print(response)
    try:
        numbers = re.findall(r'\d+', response.choices[0]["message"]["content"])
        print(numbers)
        score = int(numbers[0])
        if 0 <= score <= 10:
            return score
        else:
            return "The score generated by the model is out of range. Please try again."
    except ValueError:
        return "The model did not generate a valid score. Please try again."

input = df.iloc[0:10]["Question"].to_list()
score = score_by_openai_2(input)
print(f"score: {score}")

InvalidRequestError: ignored

In [ ]:
response = openai.ChatCompletion.create(model = "gpt-3.5-turbo",messages = [{"role": "user", "content": "I have a sentence, and I want to get the log_prob of chatgpt to generate it. How should I write the code"}],temperature = 1,max_tokens=1000)

In [ ]:
print(response.choices[0]["message"]["content"])

To get the log probability of ChatGPT generating a specific sentence, you can use the `ChatCompletion` class from the OpenAI API. Here's an example of how you can write the code in Python:

```python
import openai

def get_log_probability(prompt, text):
    # Set up OpenAI API credentials
    openai.api_key = 'YOUR_API_KEY'

    # Define your prompt
    prompt_text = "Translate the following English text to French:"

    # Set up the completion request
    completion = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt_text,
        max_tokens=100,
        logprobs=0,  # Set to 1 to retrieve log probabilities
        temperature=0.8,
        n=1,
        stop=None,
        temperature=0.7,
        log_level="info"
    )

    # Extract the log probability for the generated text
    log_prob = completion.choices[0].logprobs.token_logprobs[-1]

    return log_prob
```

In this code, you need to replace `'YOUR_API_KEY'` with your actual OpenAI API key. The `

In [ ]:
import openai

# Set up your OpenAI API credentials
# openai.api_key = "YOUR_API_KEY"

# Define the prompt sentence
prompt = "Your sentence goes here."

# Generate completion using ChatGPT
response = openai.Completion.create(
  engine="text-davinci-003",  # Replace with your preferred engine
  prompt=prompt,
  max_tokens=50,  # Adjust as needed
  n=1,
  stop=None,
  temperature=0.0,  # Set temperature to 0.0 for deterministic output
  logprobs=0  # Set logprobs to 0 to disable log probability tracking
)
response

<OpenAIObject text_completion id=cmpl-7rHfauJNCdSeFgarvUvNwAKGfD3F6 at 0x79c335024720> JSON: {
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-7rHfauJNCdSeFgarvUvNwAKGfD3F6",
  "object": "text_completion",
  "created": 1692933662,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\nI'm excited to start this new adventure!",
      "index": 0,
      "logprobs": {
        "tokens": [
          "\n",
          "\n",
          "I",
          "'m",
          " excited",
          " to",
          " start",
          " this",
          " new",
          " adventure",
          "!"
        ],
        "token_logprobs": [
          -0.004667344,
          -0.0029592365,
          -0.13739866,
          -1.2596819,
          -1.2240975,
          -0.03292883,
          -0.46789134,
          -0.37497002,
          -0.015200004,
          -0.751

In [ ]:
len(response["choices"][0]["logprobs"]["tokens"]), len(response["choices"][0]["logprobs"]["token_logprobs"])

(11, 11)

In [ ]:
import requests
import json

# def get_log_prob(prompt):
# Define your API endpoint
endpoint = "https://api.openai.com/v1/chat/completions"

# Set your OpenAI API key
api_key = "sk-"

# Set the prompt for which you want to obtain the log probability
prompt = "I have a sentence."
data = {
    "model": "gpt-3.5-turbo",
    "prompt": prompt,
    "max_tokens": 0,
    "log_level": "1"
}

# Make the API call
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
response = requests.post(endpoint, headers=headers, json=data)

# Get the log probability from the API response
log_prob = response.json()['choices'][0]['logprobs']['token_logprobs'][0]

    # return log_prob

# Example usage

print(f"Log probability of the sentence: {log_prob}")

KeyError: ignored

In [ ]:
# response = openai.Completion.create(engine='text-davinci-003',
#                     prompt="1+1=?\nOption A: 2\nOption B: 3\n Your Answer:",
#                     temperature=0,
#                     logprobs=5,
#                     stop='\n',
#                     n=2
#                                     )
response = openai.ChatCompletion.create(model='gpt-3.5-turbo',
                    messages=[{"role": "user", "content": "1+1=?\nOption A: 2\nOption B: 3\n Your Answer:"}],
                    temperature=0,
                    stop='\n',
                    n=2
                                    )

In [ ]:
response

<OpenAIObject chat.completion id=chatcmpl-7reuLc7IBCOGADwrLmDg98IMoMLld at 0x7f3877b79490> JSON: {
  "id": "chatcmpl-7reuLc7IBCOGADwrLmDg98IMoMLld",
  "object": "chat.completion",
  "created": 1693022989,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Option A: 2"
      },
      "finish_reason": "stop"
    },
    {
      "index": 1,
      "message": {
        "role": "assistant",
        "content": "Option A: 2"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 27,
    "completion_tokens": 10,
    "total_tokens": 37
  }
}

In [ ]:
import os
import openai
openai.api_key = "sk-"
openai.Model.list()

<OpenAIObject list at 0x7f38833957b0> JSON: {
  "object": "list",
  "data": [
    {
      "id": "davinci",
      "object": "model",
      "created": 1649359874,
      "owned_by": "openai",
      "permission": [
        {
          "id": "modelperm-uJaD4C9nXA6tPNoBII9hcYF4",
          "object": "model_permission",
          "created": 1692634268,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ],
      "root": "davinci",
      "parent": null
    },
    {
      "id": "text-davinci-001",
      "object": "model",
      "created": 1649364042,
      "owned_by": "openai",
      "permission": [
        {
          "id": "modelperm-CDlahk1RbkghXDjtxqzXoPNo",
          "object": "model_permission",
          "created": 169091386

In [ ]:
openai.Completion.create(
                engine="text-davinci-003",
                prompt=prompt_batch,
                logprobs=top_logprobs or 1,
                **kwargs,
            )

False